# 1) Captura de Rostros desde los videos

Como primer paso se procede a enlazar la carpeta de drive que contiene los videos y las imagenes que se van a usar para entrenar al modelo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import imutils
from google.colab.patches import cv2_imshow

La función capturar_rostros recibe un nombre y un vídeo (mp4) como parametros. Crear una carpeta con el nombre de la persona y guarda todos los rostros detectados en cada frame del video



In [ ]:
def capturar_rostros(url_video,nombre_persona,rotacion,cantidad_rostros,nro_video):

  #En ruta_datos se guarda la ruta de la carpeta donde se almacenaran los rostros para entrenar al modelo  
  ruta_datos = '/content/drive/My Drive/Datos Diplomado/Aplicacion Rostros/Personas'
  #En ruta persona se guarda la ruta donde se almacenaran los datos de la persona especifica
  ruta_persona = ruta_datos + '/' + nombre_persona
  #Se verifica si existe la ruta de la persona sino se crea la ruta
  if not os.path.exists(ruta_persona):
	  print('Carpeta creada: ',ruta_persona)
	  os.makedirs(ruta_persona)
  
  #Este codigo es para realizar un video en directo
  #video = cv2.VideoCapture(0,cv2.CAP_DSHOW)

  #Se carga el video que se indica en url_video
  video = cv2.VideoCapture(url_video)
  #se usa el algoritmo CascadeClassifier para detectar los rostros
  clasificador = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
  #contador de rostros detectados
  nro_rostros = 0
  #recuperar cuadro por cuadro del video
  while True:
      ret, cuadro = video.read()
      if ret == False: break
      #se ha detectado que algunos videos se cargan rotados 90 o 180 grados por lo que habria que girar la imagen
      cuadro =  cv2.rotate(cuadro,rotacion)
      #Se redimensiona el ancho del fotograma del video
      cuadro =  imutils.resize(cuadro, width=640)
      #Se obtiene un fotograma(cuadro) en escala de grises
      imagen_gris = cv2.cvtColor(cuadro, cv2.COLOR_BGR2GRAY)
      cuadroaux = cuadro.copy()
      #se detectan todos los rostros en la imagen en gris
      rostros = clasificador.detectMultiScale(imagen_gris,1.3,5)
      # Por cada rostro detectado se dibuja unrectangulo
      for (x,y,w,h) in rostros:
          cv2.rectangle(cuadro, (x,y),(x+w,y+h),(0,255,0),2)
          #se recorta el rostro de la imagen, se redimensiona para que todos tengan el mismo tamaño
          #y se guarda como un archivo jpg (rostro_<nro_rostor>.jpg)
          rostro = cuadroaux[y:y+h,x:x+w]
          rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
          cv2.imwrite(ruta_persona + '/rostro_'+str(nro_video)+'_'+str(nro_rostros)+'.jpg',rostro)
          nro_rostros = nro_rostros + 1
      #cada 60 rostros se muestra el rostro capturado
      if (nro_rostros % 60 == 0) and nro_rostros>1 :
          cv2_imshow(cuadro)
      k =  cv2.waitKey(1)
      #El proceso termina cuando se consiguio la cantidad de rostros solicitada
      if k == 27 or nro_rostros >= cantidad_rostros:
          break
  video.release()
  cv2.destroyAllWindows()

# 2) Métodos para entrenar el reconocedor

Los metodos usados son los siguientes:

---
**EigenFaceRecognizer** : Este emplea el método PCA (Principal Component Analysis), Según la documentación de OpenCV es: 

> *La idea es que un conjunto de datos de alta dimensión a menudo se describe mediante variables correlacionadas y, por lo tanto, solo unas pocas dimensiones significativas explican la mayor parte de la información.*

Consideraciones:

*   Las imágenes de entrenamiento como de predicción deben estar en escala de grises.
*   El método eigenfaces asume que todas las imágenes, ya sean de entrendamiento o test deben tener el mismo tamaño.
---
**FisherFaceRecognizer** : Este método es una mejora de EigenFaceRecognizer 

Consideraciones:

*   Las imágenes de entrenamiento como de predicción deben estar en escala de grises.
*   El método eigenfaces asume que todas las imágenes, ya sean de entrendamiento o test deben tener el mismo tamaño.
---
**BPH (Histogramas de Patrones Binarios Locales)** : Presenta mejoras respecto a los métodos anteriores, ya que es más robusto ante cambios de iluminación. Además, citando a la documentación de OpenCV: 

> *La idea es no mirar la imagen completa como un vector de alta dimensión, sino describir solo las características locales de un objeto*

Consideraciones:

*   Las imágenes de entrenamiento como de predicción deben estar en escala de grises.
*   No se tiene especificaciones sobre el tamaño de las imágenes correspondientes a los rostros. Por lo tanto asumimos que pueden tener distinto tamaño.
--- 

In [ ]:
def entrenar_modelo(modelo='LBPHFace'):
	#import cv2
	#import os
	import numpy as np

	#modulo=1
	if modelo=='EigenFaces':
		modulo=50
	elif modelo=='FisherFace':
		modulo=1
	else:
		modulo=10

	#ruta_datos es la carpeta raiz donde estan todas las carpetas que almacenan los rostros para utilizar en el modelo
	ruta_datos = '/content/drive/My Drive/Datos Diplomado/Aplicacion Rostros/Personas' 
	#se crea una lista usando todas las carpetas de personas
	lista_personas = os.listdir(ruta_datos)
	print('Lista de personas: ', lista_personas)

	#En etiquetas en este se almacenarán las etiquetas correspondientes a cada imagen según la persona.
	etiquetas = []
	# en rostros[] se almacenará cada una de las imágenes de los rostros.
	rostros = []
	#identifica la persona que se esta evaluando
	persona = 0

	#Con este for leemos cada directorio asignado a cada persona 
	for persona_directorio in lista_personas:
		ruta_persona = ruta_datos + '/' + persona_directorio
		print('Leyendo las imágenes')
		contador=0
	  #con este for leemos cada archivo de un directorio
		for nombre_archivo in os.listdir(ruta_persona):
			if contador % modulo == 0:
					print('Rostros: ', persona_directorio + '/' + nombre_archivo)
	    		#en etiquetas añadimos la etiqueta correspondiente a la persona
					etiquetas.append(persona)
					#en rostros añadimos cada imagen de rostro de esa persona	
					rostros.append(cv2.imread(ruta_persona+'/'+nombre_archivo,0))
			contador=contador+1
  	#se incrementa para guardar los datos de la siguiente persona
		persona = persona + 1

	# se muestra las etiquetas (rostros) grabados por cada persona
	print('etiquetas= ',etiquetas)
	for x in range(len(lista_personas)):
		print('Número de etiquetas :'+str(x)+'',np.count_nonzero(np.array(etiquetas)==x))

  # se selecciona el modelo de acuerdo al parametro enviado
	if modelo=='EigenFaces':
		reconocedor_rostro = cv2.face.EigenFaceRecognizer_create()
	elif modelo=='FisherFace':
		reconocedor_rostro = cv2.face.FisherFaceRecognizer_create()
	else:
		reconocedor_rostro = cv2.face.LBPHFaceRecognizer_create()

	# se procede a entrenar el modelo enviandoles los rostros con su respectiva etiqueta
	print("Entrenando...")
	reconocedor_rostro.train(rostros, np.array(etiquetas))

	# se almacena el modelo seleccionado

	if modelo=='EigenFaces':
		reconocedor_rostro.write('modeloEigenFace.xml')
	elif modelo=='FisherFace':
		reconocedor_rostro.write('modeloFisherFace.xml')
	else:
		reconocedor_rostro.write('modeloLBPHFace.xml')

	print("Modelo almacenado...")


In [ ]:
entrenar_modelo('LBPHFace')
entrenar_modelo('FisherFace')
entrenar_modelo('EigenFaces')

# 3) Reconocimientos de Rostros

En este codigo se llama al modelo seleccionado para detectar un rostro conocido. Para detectar un mostro se llama a la funcion result.

Esta función nos devuelve la etiqueta relacionada con la persona y un valor de confianza el cual varia de acuerdo al modelo.

Se define un umbral por modelo y si el valor de confianza se encuentra por debajo de ese umbral se reconoce que el rostro detectado es correcto y corresponde a la etiqueta devuelta por el metodo predict, en caso contrario se etiqueta el rostro como desconocido

Los valores de confianza que se ha definido por defecto para esta aplicacion son los siguientes:

*   EigenFaces = 5700
*   FisherFace = 500
*   LBPHFace = 70




In [ ]:
def BuscarImagen(imagen,modelo='LBPHFace',umbral=70):
  import cv2
  import os
  import requests
  import base64
  #se especifica la carpeta donde estan los rostros de las personas y en base a esas carpetas
  #se crea una lista de personas
  ruta_datos = '/content/drive/My Drive/Datos Diplomado/Aplicacion Rostros/Personas' 
  lista_personas = os.listdir(ruta_datos)

  #se especifica el modelo que se va a usar
  if modelo=='FisherFace':
    reconocedor_rostro = cv2.face.FisherFaceRecognizer_create()
  elif modelo=='EigenFaces':
    reconocedor_rostro = cv2.face.EigenFaceRecognizer_create()
  else:
    reconocedor_rostro = cv2.face.LBPHFaceRecognizer_create()

  # se lee el modelo almacenado
  if modelo=='FisherFace':
    reconocedor_rostro.read('modeloFisherFace.xml')
  elif modelo=='EigenFaces':
    reconocedor_rostro.read('modeloEigenFace.xml')
  else:
    reconocedor_rostro.read('modeloLBPHFace.xml')

  # se llama al algoritmo para detectar rostros y se inicializa el contador de rostros
  detector_rostro = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
  contador_encontro=0
  contador_no_encontro=0

  #se crea una version de la imagen en escala de grises
  gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen_aux = gris.copy()

  #en rostros se almacena todos los rostros encontrados en la imagen
  # se inicializa encontro en false, si se detecta un rostro conocido encontro se gabrara como True
  rostros = detector_rostro.detectMultiScale(gris,1.3,5)
  encontro=False

  #por cada rostro encontrado    
  for (x,y,w,h) in rostros:
          #se recorta y redimensiona el rostro
          rostro = imagen_aux[y:y+h,x:x+w]
          rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)

          #se envia el rostro al algoritmo de prediccion y se recibe un resultado
          resultado = reconocedor_rostro.predict(rostro)
          #se graban los valores del resultado en la imagen
          cv2.putText(imagen,'{}'.format(resultado),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
          
          #si la distancia se encuentra por debajo del umbral se reconoce que si es la persona
          #y se graba su nombre en la imagen, se dibuja un cuadrado 
          #y se especifica que encontro a la persona
          if resultado[1] < umbral:
              cv2.putText(imagen,'{}'.format(lista_personas[resultado[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
              cv2.rectangle(imagen, (x,y),(x+w,y+h),(0,255,0),2)
              encontro=True
              contador_encontro=contador_encontro+1
          #si la distancia supera el umbral, se muestra el nombre como desconocido
          #y se dibuja un cuadrado rojo
          else:
              cv2.putText(imagen,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
              cv2.rectangle(imagen, (x,y),(x+w,y+h),(0,0,255),2)
              contador_no_encontro=contador_no_encontro+1

  if (contador_encontro>0) or (contador_no_encontro>0):
  
          cv2.imwrite("imagen.jpg",imagen)
          i1 = "/content/imagen.jpg"
          #codifica la imagen
          with open(i1, "rb") as img_file:
            encoded_string= base64.b64encode(img_file.read()).decode('utf-8')
          if encontro==True:
            r = {"encotro": True, "nombre": format(lista_personas[resultado[0]]), "imagen": encoded_string}
            return r
          else:
            r = {"encotro": True, "nombre": "Desconocido", "imagen": encoded_string}
            return r            

  r = {"encotro": False, "nombre": "", "imagen": ""}
  return r

# 4) Servidor Flask

In [ ]:
!pip install flask-ngrok 


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import base64
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import cv2

In [ ]:
#inicia la aplicacion en Flask
app = Flask(__name__)
run_with_ngrok(app)   #inicia ngrok cuando la app esté corriendo
@app.route("/who_face", methods=['GET', 'POST'])
def who_face():
    #inicializa la respuesta por defecto
    response = {"encotro": False, "nombre": "", "imagen": ""}
    
    #recupera la informacion desde la aplicacion
    content = request.get_json(silent=True)
    
    #si no hay contenido devuelve la respuesta por defecto
    if not content:
      return jsonify(response)

    #si encuentra que se envio la imagen
    if content['imagen'] != '':

      #decodifica la imagen, la transforma en una matriz y la carga con
      #la libreria cv2
      imgdata = base64.b64decode(content['imagen'])
      npimg = np.fromstring(imgdata, dtype=np.uint8)
      ProcessImage = cv2.imdecode(npimg, 1)
  
      #muestra la imagen y los datos recibidos desde la app    
      cv2_imshow(ProcessImage)
      print(content['algoritmo'])
      print(content['confianza'])
      

      #envia la imagen recibida al algoritmo que se encargara de detectar si
      #hay una persona en la imagen y devuelve esa imagen codificada
      imgEncontrada = BuscarImagen(ProcessImage,content['algoritmo'],int(content['confianza']))

      #se reenvia la respuesta a la App
      response['imagen'] = imgEncontrada['imagen']
      response['nombre'] = imgEncontrada['nombre']
      response['encotro'] = imgEncontrada['encotro']


    print(response, content)
    return jsonify(response)
app.run()
